## Notebook to visualize colocalization results as a heatmap

In [ ]:
!date

#### import libraries

In [ ]:
from pandas import read_csv, concat, pivot, read_pickle
from pandas.errors import EmptyDataError
import matplotlib.pyplot as plt
from matplotlib.pyplot import rc_context
from seaborn import heatmap
from os.path import exists
from math import ceil
import numpy as np

%matplotlib inline
# for white background of figures (only for docs rendering)
%config InlineBackend.print_figure_kwargs={'facecolor' : "w"}
%config InlineBackend.figure_format='retina'

#### set notebook variables

In [ ]:
# naming
cohort = 'foundin'
dx = 'PD'

# directories
wrk_dir = '/labshare/raph/datasets/foundin_qtl'
results_dir = f'{wrk_dir}/results'
figures_dir = f'{wrk_dir}/figures'
quants_dir = f'{wrk_dir}/quants'
public_dir = f'{wrk_dir}/public'

# in file
rnab_features_file = f'{public_dir}/gencode_v29.lncipedia_v5_2_hc.annotation.pkl'
pdui_features_file = f'{quants_dir}/{cohort}_PDUI.features.csv'

# out files
figure_file = f'{figures_dir}/{cohort}.colocalization.{dx}.png'

# variables
DEBUG = False
days = ['da0', 'da25', 'da65', 'daNA', 'D52']
modalities = ['PDUI', 'RNAB', 'RNAS', 'CIRC', 'SCRN-DA', 'SCRN-ElC', 
              'SCRN-eNP', 'SCRN-iDA', 'SCRN-lNP', 'SCRN-NlC', 'SCRN-PFPP', 
              'PDUI-DA', 'PDUI-iDA',
              'Bryois-Astro', 'Bryois-Endo', 'Bryois-ExN', 'Bryois-InN', 
              'Bryois-Micro', 'Bryois-OPC', 'Bryois-Oligo', 'Bryois-Peri', 
              'Bulk-meta', 'DAn-meta', 'MetaBrain-basalganglia', 
              'MetaBrain-cerebellum', 'MetaBrain-cortex', 
              'MetaBrain-hippocampus', 'MetaBrain-spinalcord', 
              'hipsci-DA', 'hipsci-pseudobulk']
dpi_value = 100
min_h4 = 0.5

### for each day and modality load the colocalization results

In [ ]:
coloc_df = None
for day in days:
    for modality in modalities:
        print(day, modality, end=':')
        in_file = f'{results_dir}/{cohort}_{day}_{modality}_{dx}.coloc.pp.csv'
        if exists(in_file):
            try:
                this_df = read_csv(in_file)
            except EmptyDataError:
                print(f'empty file for {in_file}')
                continue
            print(f'loaded {this_df.shape[0]} results')
            # add day and modality
            this_df['day'] = day
            this_df['modality'] = modality
            coloc_df = concat([coloc_df, this_df])
print(f'\ntotal results loaded {coloc_df.shape[0]}')
if DEBUG:
    display(coloc_df.sample(5))
    display(coloc_df.day.value_counts())
    display(coloc_df.modality.value_counts())        

### load needed feature information

#### load RNAB features
RNAB features use Ensembl IDs need to map to gene names

In [ ]:
%%time
annots_df = read_pickle(rnab_features_file)
print(f'annotations shape: {annots_df.shape}')

if DEBUG:
    display(annots_df.head())

##### deference feature IDs to features names

In [ ]:
feature_annots = annots_df.loc[annots_df.gene_id.isin(coloc_df.feature)].copy()
feature_annots = feature_annots[['gene_id', 'gene_name']]
feature_annots = feature_annots.drop_duplicates()
features_dict = feature_annots.set_index('gene_id').to_dict()['gene_name']
if DEBUG:
    display(features_dict)

#### replace the RNAB features

In [ ]:
coloc_df = coloc_df.feature.replace(features_dict)
print(f'update df shape {coloc_df.shape}')
if DEBUG:
    display(coloc_df.loc[coloc_df.modality == 'RNAB'].head())

### replace the HipSci DA and pseudobulk features
they used unversioned Ensembl IDs

In [ ]:
annots_df.columns

In [ ]:
temp_df = annots_df.gene_id.str.split('.', expand=True)
annots_df['other_gene_id'] = temp_df[0]
feature_annots = annots_df.loc[annots_df.other_gene_id.isin(coloc_df.feature)].copy()
features_dict = feature_annots.set_index('other_gene_id').to_dict()['gene_name']
coloc_df.feature.replace(features_dict, inplace=True)
print(f'update df shape {coloc_df.shape}')
if DEBUG:
    display(features_dict)
    display(coloc_df.loc[coloc_df.modality == 'hipsci-DA'].head())    

#### load PDUI features

In [ ]:
%%time
annots_df = read_csv(pdui_features_file)
print(f'annotations shape: {annots_df.shape}')

if DEBUG:
    display(annots_df.head())

##### deference feature IDs to features names

In [ ]:
feature_annots = annots_df.loc[annots_df.Loci.isin(coloc_df.feature)]
feature_annots = feature_annots[['Loci', 'Gene']]
# some genes have multiple PDUI features, append to handle
multi_features = feature_annots.duplicated(subset=['Gene'], keep=False)
feature_annots.loc[multi_features, 'Gene'] = feature_annots.Gene + ':' + feature_annots.Loci
# feature_annots.drop_duplicates(inplace=True)
features_dict = feature_annots.set_index('Loci').to_dict()['Gene']
if DEBUG:
    display(features_dict)

#### replace the PDUI features

In [ ]:
coloc_df.feature.replace(features_dict, inplace=True)
print(f'update df shape {coloc_df.shape}')
if DEBUG:
    display(coloc_df.loc[coloc_df.modality == 'PDUI'].head())

### subset based on the minimum H4 variable

In [ ]:
temp = coloc_df.loc[coloc_df.H4 > min_h4]
print(f'{temp.feature.nunique()} traits will be used')
coloc_df = coloc_df.loc[coloc_df.feature.isin(temp.feature)]
print(f'results shape after filter on H4 {coloc_df.shape}')
if DEBUG:
    display(coloc_df.head())

### reshape the dataframe from long to wide

In [ ]:
# drop the cols we don't need
temp_df = coloc_df.drop(columns=['H0', 'H1', 'H2', 'H3', 'h4_supported'])
# sort not sure if this will help may have to set order in plotting explicityly
temp_df = temp_df.sort_values(by=['modality', 'day'])
# want day/modality combo's so combine
temp_df['day_modality'] = temp_df.day + '-' + temp_df.modality
# wcoloc_df = pivot(temp_df, index='feature', columns='day_modality', values='H4')
wcoloc_df = pivot(temp_df, index='feature', columns=['day', 'modality'], values='H4')
# set precision
wcoloc_df = wcoloc_df.round(2)
#replace all zeros with NaN values
wcoloc_df.replace(0, np.nan, inplace=True)
# drop rows that are all null
wcoloc_df.dropna(how='all', inplace=True)
# # sort by foundin DA neuron results
# wcoloc_df = wcoloc_df.sort_values(by=[('daNA', 'DAn-meta'), ('da65', 'SCRN-DA'), 
#                                       ('da65', 'SCRN-iDA'), 
#                                       ('daNA', 'Bulk-meta'), ('da65', 'RNAB'), 
#                                       ('da65', 'PDUI'),
#                                       ('daNA', 'Bryois-ExN'), ('daNA', 'Bryois-InN'), 
#                                       ('daNA', 'Bryois-Micro'), ('daNA', 'Bryois-Astro'), 
#                                       ('daNA', 'Bryois-Oligo')], ascending=False)
print(f'shape of wide reformated results {wcoloc_df.shape}')
if DEBUG:
    display(wcoloc_df)

### visualize the reformated data as a heatmap

In [ ]:
out_col_order = [
    ('daNA', 'Bryois-Astro'), ('daNA', 'Bryois-Endo'), 
    ('daNA', 'Bryois-ExN'), ('daNA', 'Bryois-InN'), 
    ('daNA', 'Bryois-Micro'), ('daNA', 'Bryois-OPC'),
    ('daNA', 'Bryois-Oligo'), ('daNA', 'Bryois-Peri'),    
    ('daNA', 'MetaBrain-basalganglia'), 
    ('daNA', 'MetaBrain-cerebellum'), ('daNA', 'MetaBrain-cortex'), 
    ('daNA', 'MetaBrain-hippocampus'), ('daNA', 'MetaBrain-spinalcord'), 
    ('D52', 'hipsci-pseudobulk'), ('D52', 'hipsci-DA'),
    ('daNA', 'Bulk-meta'), ('daNA', 'DAn-meta'), 
    ('da65', 'SCRN-DA'), ('da65', 'SCRN-ElC'), ('da65', 'SCRN-NlC'),
    ('da65', 'SCRN-PFPP'), ('da65', 'SCRN-eNP'), ('da65', 'SCRN-iDA'), 
    ('da65', 'SCRN-lNP'), ('da65', 'PDUI-DA'),
    ('da0', 'PDUI'), ('da25', 'PDUI'), ('da65', 'PDUI'),
    ( 'da0', 'RNAB'), ('da25', 'RNAB'), ('da65', 'RNAB')
]

In [ ]:
wcoloc_df = wcoloc_df[out_col_order]
if wcoloc_df.shape[0] > 9:
    height = 9+ceil(wcoloc_df.shape[0]/6)
else:
    height = 9
print(dx, height)        
with rc_context({'figure.figsize': (11, height), 'figure.dpi': dpi_value}):
    plt.style.use('seaborn-v0_8-bright')    
    heatmap(wcoloc_df, annot=True, linecolor='grey',
            annot_kws={"fontsize":8}, linewidths=0.05, cmap='Blues')    
    plt.title(f'Colocalization H4 for {dx} and QTL')
    plt.savefig(figure_file, dpi=dpi_value, bbox_inches='tight', 
                transparent=True, pad_inches=1)
    plt.show()

### visualize colocalization heatmap by modality and sorted

In [ ]:
oi_modalities = [('daNA', 'DAn-meta'), ('daNA', 'Bulk-meta'), 
                 ('da65', 'PDUI'), ('D52', 'hipsci-DA'),
                 ('daNA', 'Bryois-ExN'), ('daNA', 'Bryois-InN'), 
                 ('daNA', 'Bryois-Micro'), ('daNA', 'Bryois-Astro'), 
                 ('daNA', 'Bryois-Oligo')]
for modality in oi_modalities:
    print(modality[1])
    temp_df = wcoloc_df.loc[wcoloc_df[modality] > min_h4].sort_values(modality, ascending=False)
    # sort by foundin DA neuron results
    temp_df = temp_df.sort_values(by=[modality], ascending=False)
    print(f'{temp_df.shape}')
    sub_figure_file = f'{figures_dir}/{cohort}.colocalization.{dx}.{modality[1]}.png'
    with rc_context({'figure.figsize': (11, 11), 'figure.dpi': dpi_value}):
        plt.style.use('seaborn-v0_8-bright')    
        heatmap(temp_df, annot=True, linecolor='grey',
                annot_kws={"fontsize":6}, linewidths=0.05, cmap='Blues')
        plt.yticks(rotation=0)
        plt.title(f'Colocalization H4 for {dx} and QTL in {modality}')
        plt.savefig(sub_figure_file, dpi=dpi_value, bbox_inches='tight', 
                    transparent=True, pad_inches=1)
        plt.show()    

### print features with above threshold H4 by modalities of interest

In [ ]:
oi_modalities = [('daNA', 'DAn-meta'), ('daNA', 'Bulk-meta'), 
                 ('da65', 'PDUI'), ('da65', 'PDUI-DA'),
                 ('daNA', 'Bryois-ExN'), ('daNA', 'Bryois-InN'), 
                 ('daNA', 'Bryois-Micro'), ('daNA', 'Bryois-Astro'), 
                 ('daNA', 'Bryois-Oligo'), ('daNA', 'MetaBrain-basalganglia'), 
              ('daNA', 'MetaBrain-cerebellum'), ('daNA', 'MetaBrain-cortex'), 
              ('daNA', 'MetaBrain-hippocampus'), ('daNA', 'MetaBrain-spinalcord'), 
                 ('D52', 'hipsci-DA'), ('D52', 'hipsci-pseudobulk')]
for modality in oi_modalities:
    print(modality)
    temp = wcoloc_df.loc[wcoloc_df[modality] > min_h4].sort_values(modality, ascending=False)
    print(temp.index)
    display(temp[modality])

In [ ]:
!date